# Trabalho 1 - MC853B

**Alunos:** 

Lucca Miranda Nunes - 230554

Luiz Felipe Corradini Rego Costa - 230613

Pablo Henrique Almeida Mendes - 230977




**Doença de notificação compulsória escolhida:** Esquistossomose

****

## 1) Upload e investigação dos dados

### 1.1) Análise Exploratória dos dados

Os dados escolhidos para treinamento foram os da doença esquistossomose dos anos de 2020, 2021, 2022 e 2023. Os anos de 2024 em diante não estavam disponibilizados na base do SINAN. Todos os dados estão no diretório source/csv.

In [403]:
import pandas as pd

esq_19 = pd.read_csv('source/csv/ESQUBR19.csv')
esq_20 = pd.read_csv('source/csv/ESQUBR20.csv')
esq_21 = pd.read_csv('source/csv/ESQUBR21.csv')
esq_22 = pd.read_csv('source/csv/ESQUBR22.csv')
esq_23 = pd.read_csv('source/csv/ESQUBR23.csv')

In [404]:
print(esq_19.shape)
print(esq_20.shape)
print(esq_21.shape)
print(esq_22.shape)
print(esq_23.shape)

(3608, 54)
(1873, 54)
(2395, 54)
(2640, 54)
(3059, 54)


Criando uma coluna para identificar o ano de cada notificação:

In [405]:
esq_19['Ano_atual'] = 2019
esq_20['Ano_atual'] = 2020
esq_21['Ano_atual'] = 2021
esq_22['Ano_atual'] = 2022
esq_23['Ano_atual'] = 2023

In [406]:
esq_23.columns

Index(['Unnamed: 0', 'TP_NOT', 'ID_AGRAVO', 'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO',
       'SG_UF_NOT', 'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'DT_SIN_PRI',
       'SEM_PRI', 'ANO_NASC', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA',
       'CS_ESCOL_N', 'SG_UF', 'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS',
       'DT_INVEST', 'ID_OCUPA_N', 'DT_COPRO', 'AN_QUANT', 'OUTROS', 'TRATAM',
       'TRATANAO', 'STCURA1', 'STCURA2', 'STCURA3', 'FORMA', 'TPAUTOCTO',
       'COUFINF', 'COPAISINF', 'COMUNINF', 'NOPROPIN', 'NOCOLINF',
       'DOENCA_TRA', 'EVOLUCAO', 'DT_ENCERRA', 'DT_DIGITA', 'DT_TRANSUS',
       'DT_TRANSDM', 'DT_TRANSSM', 'DT_TRANSRS', 'DT_TRANSSE', 'DT_OBITO',
       'DS_FORMA', 'AN_QUALI', 'DTTRAT', 'DT_RESU3', 'OUTRO_EX', 'Ano_atual'],
      dtype='object')

Precisamos investigar a quantidade de atributos-alvo em cada ano, a fim de garantir balanceamento nas classes dentro dos datasets de treino e de teste:

Valores correspondentes a cada desfecho possível do paciente:


1 - Cura

2 - Não Cura

3 - Óbito por 
esquistossomose 

4 -  Óbito por outras 
causas 

9 - Ignorado


Como iremos usar os dados dos três anos, faz mais sentido concatená-los em um único DataFrame, a fim de realizar a análise em um único comando

In [407]:
esq_total = pd.concat([esq_19, esq_20, esq_21, esq_22, esq_23])
print(esq_total['EVOLUCAO'].value_counts())

# Reajustando os índices
esq_total.reset_index(drop=True, inplace=True)

1.0    7081
9.0    1452
3.0     330
2.0     273
4.0     142
Name: EVOLUCAO, dtype: int64


Como o problema que iremos prever é de natureza binária (óbito ou não-óbito), precisamos remover todas as instâncias que a resposta foi "Ignorado" ou "Não-Cura". Se o caso foi ignorado, não podemos inserí-lo em nenhuma das duas classes, e a classificação de "Não-Cura" também não torna possível avaliar se o paciente veio a óbito ou não. 

Com isso, levaremos em consideração também os pacientes que vieram a óbito por outros casos no peródo de tempo além da doença.

In [408]:
esq_total = esq_total[(esq_total['EVOLUCAO'] == 1) | (esq_total['EVOLUCAO'] == 3) | (esq_total['EVOLUCAO'] == 4)]
print(esq_total['EVOLUCAO'].value_counts())
print(esq_total.shape)

1.0    7081
3.0     330
4.0     142
Name: EVOLUCAO, dtype: int64
(7553, 55)


In [409]:
print(f"Porcentagem de mortes por esquistossomose: {len(esq_total[esq_total['EVOLUCAO'].isin([3,4])]) / len(esq_total) * 100 :.2f}%")

Porcentagem de mortes por esquistossomose: 6.25%


Inicialmente, pensamos em filtrar as doenças pela que a data de notificação e a data dos primeiros sintomas tenha diferença de até 40 dias. Porém, como a esquistossomose é uma doença que também apresenta uma fase crônica (que pode durar até 6 meses)[FONTE], então optamos por não utilizar esse critério como excludente de instâncias no dataset. 

### 1.2) Tratamento de dados incorretos

Para garantir que não estaremos inserindo no modelo dados contraditórios, tais como o óbito ser registrado depois da data dos primeiros sintomas ou data de encerramento ser registrada antes dos primeiros sintomas.

In [410]:
# Salvando casos que a data de óbito é não-nula (ou seja, a pessoa veio a óbito)
datas_obito_para_analise = esq_total[['DT_SIN_PRI','DT_NOTIFIC', 'DT_ENCERRA','DT_OBITO','EVOLUCAO']]

datas_obito_nao_nulas = datas_obito_para_analise.filter(['DT_NOTIFIC', 'DT_OBITO','EVOLUCAO']).dropna()

# Datas sem retirar óbitos nulos 
datas_para_analise = esq_total[['DT_SIN_PRI','DT_NOTIFIC', 'DT_ENCERRA','EVOLUCAO']]

datas_nao_nulas = datas_para_analise.filter(['DT_NOTIFIC','EVOLUCAO']).dropna()

datas_obito_nao_nulas

,DT_NOTIFIC,DT_OBITO,EVOLUCAO
17,2019-01-04,2018-12-27,3.0
59,2019-01-09,2018-02-21,3.0
80,2019-01-11,2017-05-11,3.0
89,2019-01-13,2019-01-06,3.0
93,2019-01-14,2017-11-22,3.0
...,...,...,...
13423,2023-12-04,2023-06-17,3.0
13484,2023-12-12,2023-12-08,3.0
13502,2023-12-14,2023-02-11,3.0
13511,2023-12-18,2023-12-15,3.0


In [411]:

# Analisando os casos que os óbitos ocorreram antes da data de notificação
diferenca_obitos = datas_obito_nao_nulas.apply(lambda x: pd.to_datetime(x['DT_OBITO']) - pd.to_datetime(x['DT_NOTIFIC']), axis=1)
diferenca_obitos = diferenca_obitos.apply(lambda x: x.days)

diferenca_obitos

17        -8
59      -322
80      -610
89        -7
93      -418
        ... 
13423   -170
13484     -4
13502   -306
13511     -3
13512   -293
Length: 472, dtype: int64

In [412]:

obitos_negativos = diferenca_obitos[diferenca_obitos < 0]

print("Casos que o óbito foi registrado antes da notificação: ",diferenca_obitos[diferenca_obitos < 0].count())

# Analisando os casos que a data de encerramento foi registrada antes dos primeiros sintomas
diferenca_notifica = datas_para_analise.apply(lambda x: pd.to_datetime(x['DT_ENCERRA']) - pd.to_datetime(x['DT_SIN_PRI']), axis=1)
diferenca_notifica = diferenca_notifica.apply(lambda x: x.days)  
print("Casos que a data de encerramento foi registrada antes dos primeiros sintomas: ",diferenca_notifica[diferenca_notifica < 0].count())


Casos que o óbito foi registrado antes da notificação:  271
Casos que a data de encerramento foi registrada antes dos primeiros sintomas:  0


Pudemos ver que um número considerável de óbitos foram registrados depois da data de notificação, ou seja, são dados notificados incorretamente, e por isso, temos que removê-los.

In [413]:
obitos_negativos.index

Int64Index([   17,    59,    80,    89,    93,    94,   116,   128,   139,
              262,
            ...
            13245, 13246, 13254, 13277, 13330, 13423, 13484, 13502, 13511,
            13512],
           dtype='int64', length=271)

In [415]:
print(esq_total.shape)
morreu_tot = esq_total.drop(obitos_negativos.index, axis=0)
print(morreu_tot.value_counts())
print(esq_total.drop(obitos_negativos.index, axis=0).shape)

(7553, 55)
Series([], dtype: int64)
(7282, 55)


**TODO**

In [416]:
# Avaliar a janela de tempo entre o primeiro atendimento e a morte (não pode ser muito grande, em média uns 30/40 dias)
#       - remover caso esteja fora dessa janela. ver se não vão ficar muito poucos exemplos
#       - caso fiquem, podemos pegar mais anos para complementar
#       - ver também se não tem nenhum dado que a morte é antes do diagnóstico, coisas assim
# Remover dados duplicados da base. provavelmente vai ter que fazer isso antes de filtrar as colunas
# Avaliar como vai ser a divisão dos dados de treino e teste por tempo
#    - essa divisão tem que ser feita antes de fazer qualquer tipo de tratamento nos dados. depois, continua o tratamento apenas com os dados de TREINO
# Avaliar também como garantir 10% de representatividade de cada classe nos conjuntos de treino e teste (atualmente temos uns 3% de mortes)
#   - Dezin falou que tem uma técnica chamada near miss que ajuda a fazer isso, quem for puxar isso fala com ele
# Olhar outliers (que eu acho que nao precisa), plotar graficos das distribuições dos dados tambem (ela pede pro trabalho)

****

## 2) Pré-processamento dos dados

Podemos ver que, pelo número elevado de colunas, será necessário realizar uma análise quanto às colunas necessárias, para posteriormente avaliar se o total de instâncias apresentadas nos três anos serão suficientes para treinar o modelo.

### 2.1) Removendo Colunas desnecessárias

O primeiro passo para pré-processar será **remover colunas desnecessárias do dataset**. 

Depois de uma primeira análise, a equipe selecionou 11 colunas que julgamos como mais relevantes para o treinamento do modelo:


*colocar info das colunas*


In [417]:
esq_total.columns

Index(['Unnamed: 0', 'TP_NOT', 'ID_AGRAVO', 'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO',
       'SG_UF_NOT', 'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'DT_SIN_PRI',
       'SEM_PRI', 'ANO_NASC', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA',
       'CS_ESCOL_N', 'SG_UF', 'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS',
       'DT_INVEST', 'ID_OCUPA_N', 'DT_COPRO', 'AN_QUANT', 'OUTROS', 'TRATAM',
       'TRATANAO', 'STCURA1', 'STCURA2', 'STCURA3', 'FORMA', 'TPAUTOCTO',
       'COUFINF', 'COPAISINF', 'COMUNINF', 'NOPROPIN', 'NOCOLINF',
       'DOENCA_TRA', 'EVOLUCAO', 'DT_ENCERRA', 'DT_DIGITA', 'DT_TRANSUS',
       'DT_TRANSDM', 'DT_TRANSSM', 'DT_TRANSRS', 'DT_TRANSSE', 'DT_OBITO',
       'DS_FORMA', 'AN_QUALI', 'DTTRAT', 'DT_RESU3', 'OUTRO_EX', 'Ano_atual'],
      dtype='object')

In [418]:
# Lista das colunas conforme dicionário SINAN
colunas_selecionadas = [
    'CS_GESTANT',                     
    'AN_QUANT',
    'AN_QUALI',            
    'TRATAM', 
    'FORMA',      
    'CS_ESCOL_N', 
    'CS_SEXO',  
    'ANO_NASC',
    'COUFINF',
    'EVOLUCAO',
    'Ano_atual'
]

# Seleção das colunas
esq_filtrado = esq_total[colunas_selecionadas]

esq_filtrado.head()


,CS_GESTANT,AN_QUANT,AN_QUALI,TRATAM,FORMA,CS_ESCOL_N,CS_SEXO,ANO_NASC,COUFINF,EVOLUCAO,Ano_atual
0,6,0.0,1.0,1.0,1.0,4.0,M,1994.0,29.0,1.0,2019
1,6,1.0,1.0,1.0,1.0,5.0,M,1985.0,31.0,1.0,2019
3,6,0.0,3.0,1.0,1.0,9.0,M,1997.0,31.0,1.0,2019
4,6,0.0,3.0,1.0,1.0,9.0,M,1992.0,32.0,1.0,2019
5,5,1.0,1.0,1.0,1.0,0.0,F,1956.0,NaN,1.0,2019


Renomeando as colunas selecionadas para garantir melhor entendimento:

In [419]:
novos_nomes_doenças = {
    'CS_GESTANT': 'Gestante',
    'AN_QUANT': 'Quantidade ovos encontrados',
    'AN_QUALI': 'Exame_qualitativo',
    'TRATAM': 'Tratamento realizado',
    'FORMA': 'Forma clínica',
    'CS_ESCOL_N': 'Escolaridade',
    'CS_SEXO': 'Sexo',
    'ANO_NASC': 'Ano de nascimento',
    'COUFINF': 'Unidade Federativa',
    'EVOLUCAO': 'Resultado'

}

esq_filtrado = esq_filtrado.rename(columns=novos_nomes_doenças)

print(esq_filtrado.shape)
esq_filtrado.head()

(7553, 11)


,Gestante,Quantidade ovos encontrados,Exame_qualitativo,Tratamento realizado,Forma clínica,Escolaridade,Sexo,Ano de nascimento,Unidade Federativa,Resultado,Ano_atual
0,6,0.0,1.0,1.0,1.0,4.0,M,1994.0,29.0,1.0,2019
1,6,1.0,1.0,1.0,1.0,5.0,M,1985.0,31.0,1.0,2019
3,6,0.0,3.0,1.0,1.0,9.0,M,1997.0,31.0,1.0,2019
4,6,0.0,3.0,1.0,1.0,9.0,M,1992.0,32.0,1.0,2019
5,5,1.0,1.0,1.0,1.0,0.0,F,1956.0,NaN,1.0,2019


### 2.2) Selecionando os dados a serem usados

In [420]:
esq_valid = esq_filtrado[(esq_filtrado['Resultado'] == 1) | (esq_filtrado['Resultado'] == 3)]
esq_valid['Resultado'].value_counts()

1.0    7081
3.0     330
Name: Resultado, dtype: int64

In [421]:
print(f"Porcentagem de mortes por esquistossomose: {len(esq_valid[esq_valid['Resultado'] == 3]) / len(esq_valid) * 100 :.2f}%")

Porcentagem de mortes por esquistossomose: 4.45%


### 2.3) Tratamento de atributos categóricos

Primeiramente, vamos visualizar nossos dados a fim de transformar os atributos categóricos em numéricos:

In [422]:
esq_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7411 entries, 0 to 13574
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Gestante                     7411 non-null   int64  
 1   Quantidade ovos encontrados  7411 non-null   float64
 2   Exame_qualitativo            5907 non-null   float64
 3   Tratamento realizado         7411 non-null   float64
 4   Forma clínica                5662 non-null   float64
 5   Escolaridade                 6708 non-null   float64
 6   Sexo                         7411 non-null   object 
 7   Ano de nascimento            7363 non-null   float64
 8   Unidade Federativa           5973 non-null   float64
 9   Resultado                    7411 non-null   float64
 10  Ano_atual                    7411 non-null   int64  
dtypes: float64(8), int64(2), object(1)
memory usage: 694.8+ KB


Podemos notar que, apesar de que o tipo de muitos dos atributos esteja definido como 'float', de acordo com o dicionário da doença, cada número representa uma característica distinta. Por isso, a ideia será realizar o One-Hot encoding para separar tais atributos categóricos.

#### 2.3.1) Gestante

In [423]:
esq_valid['Gestante'].value_counts()

6    5206
5    2003
9     154
2      18
1      13
4      12
3       5
Name: Gestante, dtype: int64

In [424]:
# Para gestante, consideraremos apenas como uma variável binária. 
# 5, 6 e 9 indica que não é gestante, e todas as demais indicam que é gestante

esq_valid['nao_gestante'] = (esq_valid['Gestante'].isin([5, 6, 9])).astype(int)
esq_valid['gestante'] = (~esq_valid['Gestante'].isin([5, 6, 9])).astype(int)


/tmp/ipykernel_15556/433438737.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  esq_valid['nao_gestante'] = (esq_valid['Gestante'].isin([5, 6, 9])).astype(int)
/tmp/ipykernel_15556/433438737.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  esq_valid['gestante'] = (~esq_valid['Gestante'].isin([5, 6, 9])).astype(int)


In [425]:
esq_valid[esq_valid['nao_gestante'] == 0]['Resultado'].value_counts()

1.0    48
Name: Resultado, dtype: int64

Após análise do fator gestante, é possível notar que um número inexpressivo de pessoas na base de dados são gestantes, e nenhuma delas apresenta veio a óbito. Adicionar tal feature na base pode corroborar com o viés do modelo, por isso optamos por retirar a categoria da base de dados.

In [426]:
esq_valid = esq_valid.drop(columns=['Gestante', 'nao_gestante','gestante'])

#### 2.3.2) Quantidade de ovos de schistossoma encontrados

Para o atributo "Quantidade de Ovos encontrados", este já está em forma binária, indicando se não foram encontrados ovos da Schistossoma no exame de fezes (0) ou se foram encontrados (1)

In [427]:
esq_valid['Quantidade ovos encontrados'].value_counts()

0.0    3880
1.0    3531
Name: Quantidade ovos encontrados, dtype: int64

In [428]:
# Transformando o atributo de float para inteiro
esq_valid['Quantidade ovos encontrados'] = esq_valid['Quantidade ovos encontrados'].astype(int)

#### 2.3.3) Exame qualitativo

Já para o atributo "Resultado exame ovos qualitativo", que indica o resultado (1=Positivo, 2=Negativo e 3=Não realizado), como são apresentadas mais de duas categorias, será necessário transformar em mais colunas.

Além disso, como existem muitos valores nulos na coluna, vamos atribuí-los a uma quarta categoria: Desconhecido

In [429]:
print(esq_valid['Exame_qualitativo'].value_counts())
print("Valores nulos: ", esq_valid['Exame_qualitativo'].isna().sum())

1.0    3226
3.0    2090
2.0     591
Name: Exame_qualitativo, dtype: int64
Valores nulos:  1504


In [430]:
# Adicionando "Desconhecido"
esq_valid['Exame_qualitativo'].fillna("Desconhecido", inplace=True)

In [431]:
# Definindo valores numéricos como texto
esq_valid['Exame_qualitativo'].replace(1, "Positivo", inplace=True)
esq_valid['Exame_qualitativo'].replace(2, "Negativo", inplace=True)
esq_valid['Exame_qualitativo'].replace(3, "Nao_Realizado", inplace=True)

In [432]:
# Criando novas colunas com one-hot-encoding
df_onehot = pd.get_dummies(esq_valid['Exame_qualitativo'], prefix='Exame', dtype=int)

# Concatenando com o DataFrame original
esq_valid = pd.concat([esq_valid, df_onehot], axis=1)

# Removendo a coluna original
esq_valid.drop(columns=['Exame_qualitativo'], inplace=True)

#### 2.3.4) Tratamento realizado

A categoria apresenta o tratamento que foi realizado pelo paciente no combate à doença, sendo 1=  Sim  –  Praziquantel,  2=  Sim,  Oxaminiquine,  3=  Não  ou  9=  Ignorado

In [433]:
print(esq_valid['Tratamento realizado'].value_counts())
print("Valores nulos: ", esq_valid['Tratamento realizado'].isna().sum())

1.0    6256
3.0     671
9.0     454
2.0      30
Name: Tratamento realizado, dtype: int64
Valores nulos:  0


Como não existem valores nulos, podemos apenas atribuir a cada valor numérico a string relativa ao output:

In [434]:
# Substituição dos valores numéricos por rótulos
esq_valid['Tratamento realizado'].replace(
    {
        1: "Sim_Praziquantel",
        2: "Sim_Oxaminiquine", 
        3: "Nao",
        9: "Ignorado"
    }, 
    inplace=True
)

In [435]:
# Criar colunas one-hot para cada categoria
df_onehot = pd.get_dummies(
    esq_valid['Tratamento realizado'], 
    prefix='Tratamento', 
    dtype=int
)

# Concatenar com o DataFrame original
esq_valid = pd.concat([esq_valid, df_onehot], axis=1)

# Remover a coluna original (opcional)
esq_valid.drop(columns=['Tratamento realizado'], inplace=True)

#### 2.3.5) Forma clínica avaliada

In [436]:
print(esq_valid['Forma clínica'].value_counts())
print("Valores nulos: ", esq_valid['Forma clínica'].isna().sum())

1.0    4156
5.0     467
3.0     382
2.0     360
4.0     297
Name: Forma clínica, dtype: int64
Valores nulos:  1749


In [437]:
# Substituindo valores nulos para "Nao_especificado"
esq_valid['Forma clínica'].fillna("Nao_especificado", inplace=True)

In [438]:
# Dicionário de mapeamento dos valores
mapeamento_forma = {
    1: "Intestinal",
    2: "Hepato_Intestinal",
    3: "Hepato_Esplenica",
    4: "Aguda",
    5: "Outra"
}

# Substituição inplace
esq_valid['Forma clínica'].replace(mapeamento_forma, inplace=True)

In [439]:
# Criando colunas one-hot
df_onehot = pd.get_dummies(
    esq_valid['Forma clínica'],
    prefix='Forma',     
    dtype=int          
)

# Concatenar com o DataFrame original
esq_valid = pd.concat([esq_valid, df_onehot], axis=1)

# Remover a coluna original
esq_valid.drop(columns=['Forma clínica'], inplace=True)

#### 2.3.6) Escolaridade

In [440]:
print(esq_valid['Escolaridade'].value_counts())
print("Valores nulos: ", esq_valid['Escolaridade'].isna().sum())

9.0     1502
6.0     1147
3.0      869
1.0      863
5.0      520
2.0      509
4.0      507
8.0      266
0.0      213
10.0     210
7.0      102
Name: Escolaridade, dtype: int64
Valores nulos:  703


Como o campo escolaridade apresenta uma cardinalidade lógica, ou seja, o 1 = 1º a 4º série incompleta e 8 = Educação superior completa, podemos considerá-lo como um atributo numérico, e não é necessário realizar o One-Hot-Encoding.

Porém, como os atributos 9 = Ignorado e 10 = Não se aplica, transformaremos-os em 0 para garantir que não sejam considerados com uma cardinalidade maior que a Educação superior completa.

In [441]:
esq_valid['Escolaridade'].fillna(0, inplace=True)
esq_valid['Escolaridade'].replace(9, 0, inplace=True)
esq_valid['Escolaridade'].replace(10, 0, inplace=True)
esq_valid['Escolaridade'] = esq_valid['Escolaridade'].astype(int)

#### 2.3.7) Sexo

In [442]:
esq_valid['Sexo'].value_counts()

M    4459
F    2950
I       2
Name: Sexo, dtype: int64

Como temos apenas dois valores "Ignorado", podemos desconsiderá-los:

In [443]:
esq_valid['Sexo'].drop(esq_valid[esq_valid['Sexo'] == "I"].index, inplace=True)

In [444]:
esq_valid['Sexo'].replace("M", 0, inplace=True)
esq_valid['Sexo'].replace("F", 1, inplace=True)

#### 2.3.8) Ano de Nascimento

In [445]:
print(esq_valid['Ano de nascimento'].value_counts())
print("Valores nulos: ", esq_valid['Ano de nascimento'].isna().sum())

1978.0    158
1973.0    157
1976.0    154
1982.0    151
1981.0    149
         ... 
1927.0      1
1926.0      1
1906.0      1
1922.0      1
1923.0      1
Name: Ano de nascimento, Length: 104, dtype: int64
Valores nulos:  48


In [446]:
# Removendo as 31 pessoas sem um ano de nascimento definido
esq_valid.dropna(subset=['Ano de nascimento'], inplace=True)

Primeiramente, precisamos extrair a idade de cada paciente conforme o ano de nascimento. 

Como foram concatenados datasets de 4 anos diferentes, usaremos a coluna "Ano_atual" a fim de calcular a idade correta de cada paciente:

In [447]:
esq_valid['Idade'] = (esq_valid['Ano_atual'] - esq_valid['Ano de nascimento']).astype(int)

In [448]:
# Remover as colunas que não serão mais utilizadas
esq_valid.drop(columns=['Ano de nascimento', 'Ano_atual'], inplace=True)

#### 2.3.9) Unidade Federativa

Para a utilização da unidade federativa, a escolha realizada pelo grupo foi dividir por regiões ao invés de por estados, reduzindo a complexidade total do modelo. 

Para isso, é necessário identificar cada estado e atribuí-lo à região correspondente. 

In [449]:
print(esq_valid['Unidade Federativa'].value_counts())
print("Valores nulos: ", esq_valid['Unidade Federativa'].isna().sum())

31.0    3624
29.0     572
26.0     363
32.0     323
27.0     285
25.0     179
28.0     156
35.0     102
33.0      71
41.0      58
23.0      52
21.0      43
15.0      27
24.0      23
11.0      17
52.0       8
51.0       7
50.0       6
17.0       5
22.0       4
53.0       3
42.0       3
43.0       3
13.0       2
Name: Unidade Federativa, dtype: int64
Valores nulos:  1427


Como temos uma quantidade significativa de valores nulos, os definiremos como "Não especificado"

In [450]:
esq_valid['Unidade Federativa'].fillna("Nao_especificado", inplace=True)

Para identificar as fontes, foi preciso acessar a Tabela de Códigos de Municípios do IBGE:

https://www.ibge.gov.br/explica/codigos-dos-municipios.php

In [451]:
replace_uf_para_regiao = {
    11: 'Norte',  # Rondônia
    12: 'Norte',  # Acre
    13: 'Norte',  # Amazonas
    14: 'Norte',  # Roraima
    15: 'Norte',  # Pará
    16: 'Norte',  # Amapá
    17: 'Norte',  # Tocantins
    
    21: 'Nordeste',  # Maranhão
    22: 'Nordeste',  # Piauí
    23: 'Nordeste',  # Ceará
    24: 'Nordeste',  # Rio Grande do Norte
    25: 'Nordeste',  # Paraíba
    26: 'Nordeste',  # Pernambuco
    27: 'Nordeste',  # Alagoas
    28: 'Nordeste',  # Sergipe
    29: 'Nordeste',  # Bahia
    
    31: 'Sudeste',  # Minas Gerais
    32: 'Sudeste',  # Espírito Santo
    33: 'Sudeste',  # Rio de Janeiro
    35: 'Sudeste',  # São Paulo
    
    41: 'Sul',  # Paraná
    42: 'Sul',  # Santa Catarina
    43: 'Sul',  # Rio Grande do Sul
    
    50: 'Centro-Oeste',  # Mato Grosso do Sul
    51: 'Centro-Oeste',  # Mato Grosso
    52: 'Centro-Oeste',  # Goiás
    53: 'Centro-Oeste'   # Distrito Federal
}

In [452]:
# Substituindo os códigos pelas regiões
esq_valid['Unidade Federativa'] = esq_valid['Unidade Federativa'].replace(replace_uf_para_regiao)

# Verificando o resultado
print(esq_valid['Unidade Federativa'].unique())

['Nordeste' 'Sudeste' 'Nao_especificado' 'Sul' 'Centro-Oeste' 'Norte']


Agora, definindo uma coluna por One-Hot encoding para cada região distinta:

In [453]:
# Criando colunas one-hot
df_onehot = pd.get_dummies(
    esq_valid['Unidade Federativa'],
    prefix='Regiao',     
    dtype=int          
)

# Concatenando com o DataFrame original
esq_valid = pd.concat([esq_valid, df_onehot], axis=1)

# Removendo a coluna original
esq_valid.drop(columns=['Unidade Federativa'], inplace=True)

In [454]:
esq_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7363 entries, 0 to 13574
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Quantidade ovos encontrados  7363 non-null   int64  
 1   Escolaridade                 7363 non-null   int64  
 2   Sexo                         7363 non-null   object 
 3   Resultado                    7363 non-null   float64
 4   Exame_Desconhecido           7363 non-null   int64  
 5   Exame_Nao_Realizado          7363 non-null   int64  
 6   Exame_Negativo               7363 non-null   int64  
 7   Exame_Positivo               7363 non-null   int64  
 8   Tratamento_Ignorado          7363 non-null   int64  
 9   Tratamento_Nao               7363 non-null   int64  
 10  Tratamento_Sim_Oxaminiquine  7363 non-null   int64  
 11  Tratamento_Sim_Praziquantel  7363 non-null   int64  
 12  Forma_Aguda                  7363 non-null   int64  
 13  Forma_Hepato_Espl

In [455]:
print(esq_valid['Regiao_Norte'].value_counts())
print(esq_valid['Regiao_Nordeste'].value_counts())
print(esq_valid['Regiao_Sudeste'].value_counts())
print(esq_valid['Regiao_Sul'].value_counts())
print(esq_valid['Regiao_Centro-Oeste'].value_counts())

0    7312
1      51
Name: Regiao_Norte, dtype: int64
0    5686
1    1677
Name: Regiao_Nordeste, dtype: int64
1    4120
0    3243
Name: Regiao_Sudeste, dtype: int64
0    7299
1      64
Name: Regiao_Sul, dtype: int64
0    7339
1      24
Name: Regiao_Centro-Oeste, dtype: int64


Pudemos ver que, como a distribuição por região dos casos se mostra desigual, não seria correto realizar a divisão de treino por região. Por isso, optamos por realizar a divisão em função de tempo.